In [2]:
import math
import cv2
import numpy as np
from time import time
import mediapipe as mp
import matplotlib.pyplot as plt

In [3]:
mp_pose = mp.solutions.pose

pose = mp_pose.Pose(
    static_image_mode = False,
    min_detection_confidence = 0.5,
    model_complexity=0
    )

mp_drawing = mp.solutions.drawing_utils

In [4]:
def getPose(image, pose):
    
    
    output = image.copy()
    
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    results = pose.process(image_rgb)
    
    height, width, _ = image.shape
    
    
    landmarks = []
    
    if results.pose_landmarks:
        
        mp_drawing.draw_landmarks(image = output,
                                 landmark_list = results.pose_landmarks,
                                  connections = mp_pose.POSE_CONNECTIONS)
        
        for landmark in results.pose_landmarks.landmark:
            
            landmarks.append((int(landmark.x*width), 
                              int(landmark.y*height),
                             (landmark.z * width)))
            
    return output, landmarks

### detect inconfidence
1- square arms (sqr_arms)

    - bicep angle for both arms
    - arm pit angle for both arms is close to zero 

2- close to body straight arms (close_arms)

    - hand is closer to hip than to elbow for both hands
    - distance between arm wrists is bigger than the distance between both hips
3- both arms at crotch (crotch_arms)

    - hand is closer to hip than to elbow for both hands
    - distance between arm wrists is smaller than the distance between both hips    

In [40]:
def get_angle(p1,p2,p3):
    x1,y1,_ = p1
    x2,y2,_ = p2
    x3,y3,_ = p3
    
    angle = math.degrees(math.atan2(y3-y2,x3-x2) - math.atan2(y1-y2,x1-x2) )
    
    if angle <0:
        
        angle += 360
        
    return angle

def get_distance(p1,p2):
    x1,y1,_ = p1
    x2,y2,_ = p2
    
    distance = math.sqrt(  ((x2-x1)**2) + ((y2-y1)**2) )
    
    return distance


In [41]:
def classify(landmarks, output):
    label = "unknowen"
   
    
    # in case not all points are found
    if(len(landmarks) < 26):
        label = "incomplete"
        return output , label
    
    
    # bicep angle
    left_elbow_angle = get_angle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value])
    
    right_elbow_angle = get_angle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value],
                                landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value])
    
    
    # armpit angel
    left_armpit_angle = get_angle(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value])
    
    right_armpit_angle = get_angle(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value],
                                landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value])
    
    
    # distances
    # wrist to elbow
    rightwrist_leftelbow_dist= get_distance(
                                landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
                                )
    rightwrist_rightelbow_dist= get_distance(
                                landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value],
                                landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
                                )
    leftwrist_leftelbow_dist= get_distance(
                                landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
                                )
    leftwrist_rightelbow_dist= get_distance(
                                landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value],
                                landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
                                )
    
    # wrist to hip
    rightwrist_righthip_dist= get_distance(
                                landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value],
                                landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
                                )
    leftwrist_lefthip_dist= get_distance(
                                landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
                                )
    
    # wrist to wrist
    rightwrist_leftwrist_dist= get_distance(
                                landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
                                )
    
    # hip to hip
    righthip_lefthip_dist= get_distance(
                                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
                                )
    
    
    # test square arms
    if(right_elbow_angle >50 and right_elbow_angle <110):
        if(left_elbow_angle > 230 and left_elbow_angle < 300):
            if(right_armpit_angle >320 or right_armpit_angle <20):
                if(rightwrist_leftelbow_dist<rightwrist_rightelbow_dist):
                    if(leftwrist_rightelbow_dist<leftwrist_leftelbow_dist):
                        if(left_armpit_angle > 320 or left_armpit_angle < 20):
                            label = "Square Arms"
    
    
    # test close to body arms
    if(rightwrist_righthip_dist < rightwrist_rightelbow_dist):
        if(leftwrist_lefthip_dist < leftwrist_leftelbow_dist):
            if(rightwrist_leftwrist_dist < righthip_lefthip_dist):  
                label = "Crotch Arms"
            else:
                label = "Close Arms"
                
                
                
                
    cv2.putText(output, '{} {}'.format(round(rightwrist_leftwrist_dist),round(righthip_lefthip_dist)), (100,470),
               cv2.FONT_HERSHEY_PLAIN, 2, (0,255,0), 2)
    
    cv2.putText(output, '{} {}'.format(round(rightwrist_leftwrist_dist),round(righthip_lefthip_dist)), (100,570),
               cv2.FONT_HERSHEY_PLAIN, 2, (0,255,0), 2)
    
    return output, label

In [45]:
video = cv2.VideoCapture(0)

cv2.namedWindow("Pose",cv2.WINDOW_NORMAL)

video.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
video.set(cv2.CAP_PROP_FRAME_HEIGHT, 960)

time1 = 0
timer_start = 0
labels = {"unknowen":0,"Close Arms":0,"Crotch Arms":0,"Square Arms":0,"incomplete":0}
label = ""
while video.isOpened():
    
    # ok holds wether the frame was read or not
    ok, frame = video.read()
    
    if not ok:
        break
    frame = cv2.flip(frame,1)
    
    height, width, _ = frame.shape
    
    frame = cv2.resize(frame, (int(width * (640/ height)), 640))
    
    frame, landmarks = getPose(frame, pose)
    
    time2 = time()
    
    if(time2 - time1)> 0:
        frames_per_second = 1.0 / (time2- time1)
        
        cv2.putText(frame,
                    "FPS: {}".format(int(frames_per_second)),
                    (100,100),
                   cv2.FONT_HERSHEY_PLAIN, 2,
                   (0,255,0), 3)
        
        frame, cur_label  = classify(landmarks , frame)
        
    time1 = time2
    
    timer_end = time()
    
    if(timer_end - timer_start >= 1):
        
        label = max(labels, key=labels.get)
        labels = {"unknowen":0,"Close Arms":0,"Crotch Arms":0,"Square Arms":0,"incomplete":0}
        timer_start = timer_end
        
    labels[cur_label] +=1
               
    if label == "unknowen":
        color  = (0,0,255)
    elif label == "incomplete":
        color = (255,0,0)
    else:
        color = (0,255,0)
    
    
    cv2.putText(frame,
                label,
                (50,50),
               cv2.FONT_HERSHEY_PLAIN, 2,
               color, 3)
    
    
    cv2.imshow("Pose", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('x'):#bdos letter x ashan a2fl el camera
        break
    
video.release()
cv2.destroyAllWindows()    
    